In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.preprocessing import MinMaxScaler
import json
import itertools
from tqdm import tqdm
from __future__ import annotations
from scipy.sparse.linalg import expm
from sklearn.linear_model import Ridge
from typing import Sequence

In [2]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    num_steps = int(tmax / dt) + 1 # +1 to include t=0
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [ ]:
def chen_deriv(state, t, a=35.0, b=3.0, c=28.0):
    """
    Computes derivatives [dx/dt, dy/dt, dz/dt] for Chen system:
      dx/dt = a*(y - x)
      dy/dt = (c - a)*x + c*y - x*z
      dz/dt = x*y - b*z
    """
    x, y, z = state
    dxdt = a*(y - x)
    dydt = (c - a)*x + c*y - x*z
    dzdt = x*y - b*z
    return [dxdt, dydt, dzdt]

def generate_chen_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=50.0,
    dt=0.01,
    a=35.0,
    b=3.0,
    c=28.0
):
    """
    Integrates Chen's system from 'initial_state' up to time 'tmax' with step size 'dt'.
    Returns:
      t_vals: time array of length T
      sol   : array shape [T, 3], the trajectory [x(t), y(t), z(t)]
    """
    num_steps = int(tmax / dt)
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(chen_deriv, initial_state, t_vals, args=(a, b, c))
    return t_vals, sol

In [ ]:
lambda_max = {
        "Mackey": 0.006100,
        "Lorenz": 0.905600,
        "Rossler": 0.071400,
        "Chen": 0.829600,
        "Chua": 0.428400
        }

In [ ]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [ ]:
def compute_attractor_deviation(predictions, targets, cube_size=(0.1, 0.1, 0.1)):
    """
    Compute the Attractor Deviation (ADev) metric.

    Parameters:
        predictions (numpy.ndarray): Predicted trajectories of shape (n, 3).
        targets (numpy.ndarray): True trajectories of shape (n, 3).
        cube_size (tuple): Dimensions of the cube (dx, dy, dz).

    Returns:
        float: The ADev metric.
    """
    # Define the cube grid based on the range of the data and cube size
    min_coords = np.min(np.vstack((predictions, targets)), axis=0)
    max_coords = np.max(np.vstack((predictions, targets)), axis=0)

    # Create a grid of cubes
    grid_shape = ((max_coords - min_coords) / cube_size).astype(int) + 1

    # Initialize the cube occupancy arrays
    pred_cubes = np.zeros(grid_shape, dtype=int)
    target_cubes = np.zeros(grid_shape, dtype=int)

    # Map trajectories to cubes
    pred_indices = ((predictions - min_coords) / cube_size).astype(int)
    target_indices = ((targets - min_coords) / cube_size).astype(int)

    # Mark cubes visited by predictions and targets
    for idx in pred_indices:
        pred_cubes[tuple(idx)] = 1
    for idx in target_indices:
        target_cubes[tuple(idx)] = 1

    # Compute the ADev metric
    adev = np.sum(np.abs(pred_cubes - target_cubes))
    return adev

In [ ]:
def evaluate_nrmse(all_preds, test_target, horizons):
    """
    Evaluate model performance over multiple prediction horizons for Teacher-forced Single-step Forecasting
    """
    horizon_nrmse = {}

    for horizon in horizons:
        preds = all_preds[:horizon]
        targets = test_target[:horizon]
        squared_errors = (preds - targets)**2
        variance = np.var(targets, axis=0)
        variance[variance == 0] = 1e-8 
        nrmse = np.sqrt(np.sum(squared_errors) / (horizon * variance))
        horizon_nrmse[horizon] = nrmse

    return horizon_nrmse

In [4]:
import numpy as np
from sklearn.linear_model import Ridge
from numpy.linalg import eigvals

# ---------------------------------------------------------------------
# Utility helpers
# ---------------------------------------------------------------------
def scale_spectral_radius(W, desired_radius=0.95):
    """Affine-scale W so that its spectral radius equals desired_radius."""
    eigs = eigvals(W)
    radius = np.max(np.abs(eigs))
    if radius == 0:
        raise ValueError("Spectral radius of W is zero.")
    return W * (desired_radius / radius)


def augment_state_with_squares(x):
    """Return [x, x², 1] (same convention as in CycleReservoir3D)."""
    return np.concatenate([x, x**2, [1.0]])


# ---------------------------------------------------------------------
# Hemispherically–Hierarchical Lobe Reservoir
# ---------------------------------------------------------------------
class HHLobeReservoir:
    """
    Hemispherically-Hierarchical Lobe Reservoir (HH-LR)

    • 8 anatomical modules  —  L/R × {F, P, T, O}
    • Intra-module: Watts–Strogatz small-world graphs (ring + rewire)
    • Intra-hemisphere shortcuts: distance-modulated (exp decay)
    • Inter-hemisphere bridges: sparse homotopic callosal links
    • Lobe cardinalities follow MRI volume ratio 4 : 3 : 2 : 1
    """

    # bookkeeping ------------------------------------------------------
    _LOBES  = ['F', 'P', 'T', 'O']
    _HEMIS  = ['L', 'R']
    _CENTROIDS = {                             # rough 2-D montage coords
        ('L', 'F'): (-1.0,  1.0), ('L', 'P'): (-1.0,  0.0),
        ('L', 'T'): (-1.0, -1.0), ('L', 'O'): (-1.0, -2.0),
        ('R', 'F'): ( 1.0,  1.0), ('R', 'P'): ( 1.0,  0.0),
        ('R', 'T'): ( 1.0, -1.0), ('R', 'O'): ( 1.0, -2.0),
    }

    # ------------------------------------------------------------------
    def __init__(self,
                 reservoir_size=800,
                 input_dim=32,
                 spectral_radius=0.9,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 # small-world / shortcut hyper-parameters
                 k_ring=8,
                 p_rewire_frontal=0.30,
                 p_rewire_other=0.10,
                 P_lat=0.04,
                 sigma=5.0,
                 P_call=0.01,
                 seed=42):
        """
        reservoir_size : total number of neurons (N)
        input_dim      : dimension of u_t  (128 for EEG band-power features)
        leaking_rate   : α in leaky-integrator update
        """
        self.N          = reservoir_size
        self.D_in       = input_dim
        self.rho        = spectral_radius
        self.in_scale   = input_scale
        self.alpha      = leaking_rate
        self.ridge_alpha= ridge_alpha
        self.seed       = seed

        # --------------------------------------------------------------
        # 1) allocate neurons:  per-hemisphere 4 : 3 : 2 : 1 (F:P:T:O)
        # --------------------------------------------------------------
        ratio = {'F': 4, 'P': 3, 'T': 2, 'O': 1}
        total_w = sum(ratio.values())          # = 10
        half_N  = self.N // 2                  # neurons per hemisphere

        counts_per_hemi = {
            l: int(half_N * ratio[l] / total_w) for l in self._LOBES
        }

        # distribute rounding residuals (largest fractional remainder first)
        residual = half_N - sum(counts_per_hemi.values())
        if residual > 0:
            remainders = sorted(
                self._LOBES,
                key=lambda l: (half_N * ratio[l] / total_w) - counts_per_hemi[l],
                reverse=True
            )
            for l in remainders[:residual]:
                counts_per_hemi[l] += 1

        # build slice table
        self._module_slices = {}
        idx0 = 0
        for h in self._HEMIS:          # L then R
            for l in self._LOBES:      # F, P, T, O
                n = counts_per_hemi[l]
                self._module_slices[(h, l)] = slice(idx0, idx0 + n)
                idx0 += n
        assert idx0 == self.N, "Slice allocation error"

        # --------------------------------------------------------------
        # 2) construct adjacency matrix W according to HH-LR rules
        # --------------------------------------------------------------
        rng = np.random.default_rng(self.seed)
        W = np.zeros((self.N, self.N), dtype=float)

        # 2.1 intra-module small-world wiring
        for (h, l), sl in self._module_slices.items():
            n_mod = sl.stop - sl.start
            k = min(k_ring, n_mod - 1)
            p_rewire = p_rewire_frontal if l == 'F' else p_rewire_other

            # ring lattice
            for i_local in range(n_mod):
                for m in range(1, k + 1):
                    j_local = (i_local + m) % n_mod
                    i_glob  = sl.start + i_local
                    j_glob  = sl.start + j_local
                    W[i_glob, j_glob] = rng.standard_normal()
                    W[j_glob, i_glob] = rng.standard_normal()

            # random rewiring
            for i_local in range(n_mod):
                for m in range(1, k + 1):
                    if rng.random() < p_rewire:
                        j_local_new = rng.integers(0, n_mod - 1)
                        if j_local_new >= i_local:
                            j_local_new += 1
                        i_glob = sl.start + i_local
                        j_glob_new = sl.start + j_local_new
                        w_new = rng.standard_normal()
                        W[i_glob, j_glob_new] = w_new
                        W[j_glob_new, i_glob] = rng.standard_normal()

        # 2.2 intra-hemisphere distance-weighted shortcuts
        for h in self._HEMIS:
            for l1 in self._LOBES:
                for l2 in self._LOBES:
                    if l1 == l2:
                        continue
                    sl1 = self._module_slices[(h, l1)]
                    sl2 = self._module_slices[(h, l2)]
                    d = np.linalg.norm(
                        np.array(self._CENTROIDS[(h, l1)]) -
                        np.array(self._CENTROIDS[(h, l2)])
                    )
                    p_edge = P_lat * np.exp(-d / sigma)
                    for i in range(sl1.start, sl1.stop):
                        mask = rng.random(sl2.stop - sl2.start) < p_edge
                        js = np.nonzero(mask)[0] + sl2.start
                        if js.size:
                            W[i, js] = rng.standard_normal(size=js.size)
                            W[js, i] = rng.standard_normal(size=js.size)

        # 2.3 inter-hemisphere homotopic callosal links
        for l in self._LOBES:
            sl_L = self._module_slices[('L', l)]
            sl_R = self._module_slices[('R', l)]
            for i in range(sl_L.start, sl_L.stop):
                mask = rng.random(sl_R.stop - sl_R.start) < P_call
                js = np.nonzero(mask)[0] + sl_R.start
                if js.size:
                    W[i, js] = rng.standard_normal(size=js.size)
                    W[js, i] = rng.standard_normal(size=js.size)

        # 2.4 spectral scaling to enforce ESP
        self.W = scale_spectral_radius(W, self.rho)

        # --------------------------------------------------------------
        # 3) random input weights
        # --------------------------------------------------------------
        rng = np.random.default_rng(self.seed + 1)
        self.W_in = (rng.random((self.N, self.D_in)) - 0.5) * 2.0 * self.in_scale

        # initialize state & read-out
        self.x = np.zeros(self.N)
        self.W_out = None

    # ------------------------------------------------------------------
    # ESN core methods --------------------------------------------------
    # ------------------------------------------------------------------
    def reset_state(self):
        self.x.fill(0.0)

    def _update(self, u):
        pre = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre)
        self.x = (1.0 - self.alpha) * self.x + self.alpha * x_new

    def collect_states(self, inputs, discard=100):
        self.reset_state()
        states = []
        for u in inputs:
            self._update(u)
            states.append(self.x.copy())
        return np.array(states[discard:]), np.array(states[:discard])

    # ------------------------------------------------------------------
    # read-out ----------------------------------------------------------
    # ------------------------------------------------------------------
    def fit_readout(self, train_input, train_target, discard=100):
        states_use, _ = self.collect_states(train_input, discard)
        targets_use = train_target[discard:]
        X_aug = np.vstack([augment_state_with_squares(s) for s in states_use])

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_sequence(self, inputs):
        preds = []
        self.reset_state()
        for u in inputs:
            self._update(u)
            preds.append(self.W_out @ augment_state_with_squares(self.x))
        return np.array(preds)
    
    def predict_autoregressive(self, initial_input, n_steps):
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            out = self.W_out @ augment_state_with_squares(self.x)
            preds.append(out)
            current_in = out
        return np.array(preds)


In [ ]:
# grid = {
#     "reservoir_size": [300],
#     "input_dim": [3],
#     "spectral_radius": [0.92, 0.95, 0.98],
#     "input_scale": [0.2, 0.4, 0.6, 0.8, 1.0],
#     "leaking_rate": [0.1, 0.3, 0.5, 0.7, 0.9],
#     "ridge_alpha": [1e-8],
#     "k_ring": [6, 8, 10],
#     "p_rewire_frontal": [0.2, 0.3, 0.4],
#     "p_rewire_other": [0.05, 0.10, 0.2],
#     "P_lat": [0.02, 0.04, 0.06],
#     "sigma": [3, 5, 7],
#     "P_call": [0.005, 0.01],
# }

In [ ]:
# grid = {
#     "reservoir_size": [300],
#     "input_dim": [3],
#     "spectral_radius": [0.92, 0.95, 0.98],
#     "input_scale": [1.0],
#     "leaking_rate": [0.7, 0.9],
#     "ridge_alpha": [1e-8],
#     "k_ring": [6, 8, 10],
#     "p_rewire_frontal": [0.3, 0.4],
#     "p_rewire_other": [0.1, 0.05, 0.2],
#     "P_lat": [0.04, 0.06],
#     "sigma": [3, 5, 7],
#     "P_call": [0.005, 0.01],
# }

In [ ]:
grid = {
    "reservoir_size": [300],
    "input_dim": [3],
    "spectral_radius": [0.92],
    "input_scale": [1.0],
    "leaking_rate": [0.7, 0.9],
    "ridge_alpha": [1e-8],
    "k_ring": [6, 8, 10],
    "p_rewire_frontal": [0.3, 0.4],
    "p_rewire_other": [0.1, 0.05, 0.2],
    "P_lat": [0.04, 0.06],
    "sigma": [3, 5, 7],
    "P_call": [0.005, 0.01],
}

In [ ]:
def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json", f=generate_chen_data, lambda_max=0.9, train_fracs=[0.7, 0.75, 0.8]):
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    # horizons = list(range(10, 1001, 10))
    horizons = [200, 400, 600, 800, 1000]
    

    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))
        seed_scores_vpt = []
        horizon_nrmse_all = {h: [] for h in horizons}
        # horizon_nrmse_all_open_loop = {h: [] for h in horizons}
        adev_scores = []
        # ldev_scores = []

        for initial_state in [[1.0, 1.0, 1.0], [1.0, 2.0, 3.0], [2.0, 1.5, 4.0]]:
            tmax = 250
            dt = 0.02
            t_vals, lorenz_traj = f(
                initial_state=initial_state,
                tmax=tmax,
                dt=dt
            )

            washout = 2000
            t_vals = t_vals[washout:]
            lorenz_traj = lorenz_traj[washout:]

            scaler = MinMaxScaler()
            scaler.fit(lorenz_traj)
            lorenz_traj = scaler.transform(lorenz_traj)

            T_data = len(lorenz_traj)
            for train_frac in train_fracs:
                train_end = int(train_frac * (T_data - 1))
                train_input = lorenz_traj[:train_end]
                train_target = lorenz_traj[1:train_end + 1]
                test_input = lorenz_traj[train_end:-1]
                test_target = lorenz_traj[train_end + 1:]
                n_test_steps = len(test_input)
                initial_in = test_input[0]

                for seed in np.arange(1, 6):
                    model = model_class(**params, seed=seed)
                    model.fit_readout(train_input, train_target, discard=100)
                    preds = model.predict_autoregressive(initial_in, n_test_steps)
                    # preds_open_loop = model.predict_open_loop(test_input)

                    T_VPT_s, _, ratio = compute_valid_prediction_time(test_target, preds, t_vals, 0.4, lambda_max, dt)
                    seed_scores_vpt.append(ratio)

                    horizon_nrmse = evaluate_nrmse(preds, test_target, horizons)
                    for h in horizons:
                        horizon_nrmse_all[h].append(horizon_nrmse[h])

                    # horizon_nrmse_open_loop = evaluate_nrmse(preds_open_loop, test_target, horizons)
                    # for h in horizons:
                    #     horizon_nrmse_all_open_loop[h].append(horizon_nrmse_open_loop[h])

                    adev = compute_attractor_deviation(preds, test_target)
                    adev_scores.append(adev)

                    # ldev = compute_lyapunov_exponent("Lorenz", preds, dt)
                    # ldev_scores.append(ldev)

        mean_vpt = float(np.mean(seed_scores_vpt))
        std_vpt = float(np.std(seed_scores_vpt))
        mean_nrmse_dict = {str(h): float(np.mean(horizon_nrmse_all[h])) for h in horizons}
        std_nrmse_dict  = {str(h): float(np.std(horizon_nrmse_all[h]))  for h in horizons}
        # mean_nrmse_open_loop_dict = {str(h): float(np.mean(horizon_nrmse_all_open_loop[h])) for h in horizons}
        # std_nrmse_open_loop_dict  = {str(h): float(np.std(horizon_nrmse_all_open_loop[h]))  for h in horizons}
        mean_adev = float(np.mean(adev_scores))
        std_adev = float(np.std(adev_scores))
        # mean_ldev = float(np.mean(ldev_scores))
        # std_ldev = float(np.std(ldev_scores))

        results.append({
            "params": params,
            "seed_scores_T_VPT": seed_scores_vpt,
            "mean_T_VPT": mean_vpt,
            "std_T_VPT": std_vpt,
            "mean_NRMSEs": mean_nrmse_dict,
            "std_NRMSEs": std_nrmse_dict,
            # "mean_NRMSEs_open_loop": mean_nrmse_open_loop_dict,
            # "std_NRMSEs_open_loop": std_nrmse_open_loop_dict,
            "mean_ADev": mean_adev,
            "std_ADev": std_adev,
            # "mean_LDev": mean_ldev,
            # "std_LDev": std_ldev
        })

    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to `{output_path}`")

    return results

In [ ]:
run_grid_search(HHLobeReservoir, grid, "hhlr", output_path="hhlr_l 1.json", f=generate_lorenz_data, lambda_max=0.9)
run_grid_search(HHLobeReservoir, grid, "hhlr", output_path="hhlr_c 1.json", f=generate_chen_data, lambda_max=0.829, train_fracs=[0.7, 0.75, 0.8])


== Initial grid search for HH-LR with 1 combinations ==


Grid Search: 100%|██████████| 1/1 [00:12<00:00, 12.35s/it]


All results saved to `hhlr2.json`


[{'params': {'reservoir_size': 300,
   'input_dim': 3,
   'spectral_radius': 0.92,
   'input_scale': 1.0,
   'leaking_rate': 0.5,
   'ridge_alpha': 1e-08,
   'k_ring': 8,
   'p_rewire_frontal': 0.3,
   'p_rewire_other': 0.1,
   'P_lat': 0.04,
   'sigma': 5.0,
   'P_call': 0.01},
  'seed_scores': [np.float64(9.864),
   np.float64(8.028),
   np.float64(10.602),
   np.float64(11.970000000000004),
   np.float64(11.952),
   np.float64(8.873999999999999),
   np.float64(8.856000000000003),
   np.float64(10.674),
   np.float64(12.114),
   np.float64(8.820000000000004),
   np.float64(10.547999999999998),
   np.float64(9.306000000000003),
   np.float64(9.270000000000003),
   np.float64(11.358000000000004),
   np.float64(11.304000000000002),
   np.float64(10.709999999999999),
   np.float64(10.674),
   np.float64(10.026),
   np.float64(8.766000000000002),
   np.float64(10.782000000000004),
   np.float64(10.854000000000001),
   np.float64(12.833999999999998),
   np.float64(10.854000000000001),
   n